In [177]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

In [229]:
data = pd.read_csv("chat_dataset.csv")
data.head()

,message,sentiment
0,I really enjoyed the movie,positive
1,The food was terrible,negative
2,I'm not sure how I feel about this,neutral
3,The service was excellent,positive
4,I had a bad experience,negative


In [230]:
data1=pd.read_csv("lstm.csv")

In [231]:
data1['message']=data1['clean_text']

In [232]:
data1 = data1.drop('clean_text',axis=1)

In [165]:
dat = dat.reset_index()

In [233]:
lst=[]
for i in range(len(data)):
    if data['sentiment'][i]=='positive':
        lst.append(1.0)
    elif data['sentiment'][i]=='neutral':
        lst.append(0.0)
    else:
        lst.append(-1.0)
data['category']=lst        

In [234]:
data1.isnull().sum()

category    7
message     4
dtype: int64

In [235]:
data1=data1.dropna()


In [240]:
data = data.drop('sentiment',axis=1)
dat = pd.concat([data,data1],axis=0)

KeyError: "['sentiment'] not found in axis"

In [241]:
dat = pd.concat([data,data1],axis=0)

In [242]:
dat

,message,category
0,I really enjoyed the movie,1.0
1,The food was terrible,-1.0
2,I'm not sure how I feel about this,0.0
3,The service was excellent,1.0
4,I had a bad experience,-1.0
...,...,...
162975,why these 456 crores paid neerav modi not reco...,-1.0
162976,dear rss terrorist payal gawar what about modi...,-1.0
162977,did you cover her interaction forum where she ...,0.0
162978,there big project came into india modi dream p...,0.0


In [238]:
dat = dat.reset_index()

In [244]:
dat['message'][162979]

'have you ever listen about like gurukul where discipline are maintained even narendra modi rss only maintaining the culture indian more attack politics but someone attack hinduism rss will take action that proud for '

In [217]:
labels = pd.get_dummies(dat.category)
labels.columns = ["negative", "neutral", "positive"]
labels.head(5)

,negative,neutral,positive
0,0,0,1
1,1,0,0
2,0,1,0
3,0,0,1
4,1,0,0


In [219]:
dat = dat.drop(columns = ['category'])

In [205]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
tokenizer = Tokenizer(num_words = 8150, lower = True, split = " ", oov_token = "~")
tokenizer.fit_on_texts(dat["message"])

In [206]:
import pickle

In [207]:
word_index = tokenizer.word_index

In [245]:
word_index['won']

601

In [208]:
file = open('token1', 'wb')

# dump information to that file
pickle.dump(word_index, file)

# close the file
file.close()

In [209]:
tokenizer.word_index = word_index


In [210]:
dat["message"] = tokenizer.texts_to_sequences(dat["message"])

In [211]:
dat['message'][4]

[1877, 72, 3117, 362, 2082]

In [212]:
dat.head()

,index,message,category
0,0,"[1877, 225, 3545, 3, 506]",1.0
1,1,"[3, 936, 18, 3837]",-1.0
2,2,"[1802, 7, 243, 48, 1877, 387, 37, 9]",0.0
3,3,"[3, 1401, 18, 1357]",1.0
4,4,"[1877, 72, 3117, 362, 2082]",-1.0


In [214]:
tweets = pad_sequences(dat["message"]) 

In [215]:
tweets.shape

(163553, 52)

In [220]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tweets, labels, test_size = 0.15)

In [221]:
X_train[0].shape

(52,)

In [222]:
model = keras.models.Sequential([
    keras.layers.Embedding(input_dim = 8150, output_dim = 32),
    keras.layers.LSTM(128),
    keras.layers.Dense(128, activation = "leaky_relu", kernel_initializer = "he_normal", kernel_regularizer = "l1"),
    keras.layers.Dropout(0.35),
    keras.layers.Dense(3, activation = "softmax", kernel_initializer = "glorot_normal")
])
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 32)          260800    
                                                                 
 lstm_3 (LSTM)               (None, 128)               82432     
                                                                 
 dense_6 (Dense)             (None, 128)               16512     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 3)                 387       
                                                                 
Total params: 360,131
Trainable params: 360,131
Non-trainable params: 0
_________________________________________________________________


In [223]:
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [224]:
history = model.fit(
    X_train, y_train, 
    epochs = 5, validation_data = (X_test, y_test),
    callbacks = [keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)]
)

Epoch 1/5
4345/4345 [==============================] - 303s 69ms/step - loss: 0.8081 - accuracy: 0.8621 - val_loss: 0.2904 - val_accuracy: 0.9424
Epoch 2/5
4345/4345 [==============================] - 330s 76ms/step - loss: 0.2625 - accuracy: 0.9562 - val_loss: 0.2392 - val_accuracy: 0.9597
Epoch 3/5
4345/4345 [==============================] - 457s 105ms/step - loss: 0.2180 - accuracy: 0.9663 - val_loss: 0.2149 - val_accuracy: 0.9620
Epoch 4/5
4345/4345 [==============================] - 719s 165ms/step - loss: 0.1936 - accuracy: 0.9704 - val_loss: 0.2039 - val_accuracy: 0.9637
Epoch 5/5
4345/4345 [==============================] - 205s 47ms/step - loss: 0.1784 - accuracy: 0.9738 - val_loss: 0.2214 - val_accuracy: 0.9622


In [225]:
model.evaluate(X_test, y_test)

767/767 [==============================] - 14s 18ms/step - loss: 0.2214 - accuracy: 0.9622


[0.2214127629995346, 0.9621734023094177]

In [226]:
model.save('text_emotion1.h5')

In [227]:
model.predict([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0, 2253,  152]])

1/1 [==============================] - 1s 566ms/step


array([[0.01307813, 0.9647981 , 0.0221238 ]], dtype=float32)